In [142]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [143]:
!pip -q install folium==0.5.0
import folium

In [144]:
# !conda install -c conda-forge geocoder --yes # uncomment this line if you haven't completed the Foursquare API lab
import geocoder

In [145]:
# !conda install -c conda-forge requests --yes # uncomment this line if you haven't completed the Foursquare API lab
import requests

# Task nr 1

In [146]:
# Read in all content from website
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# Select proper dataframe with postcodes
df = df[0]

# Task nr 3

In [147]:
# Take out Boroughs that are not assigned
df = df[df["Borough"] != "Not assigned"]
# Replace Neighbourhoods with Borough name if neighborhood not assigned
df["Neighbourhood"].replace("Not assigned", df["Borough"], inplace=True)
# Sort Dataframe by Borough
df = df.sort_values(["Borough"], ascending=True).reset_index(drop=True)
# Create new dataframe to transform values of old ones
new_df = pd.DataFrame()

# Use a for loop to create the required dataframe
    # Runs through unique postcodes
for index, postcode in enumerate(df["Postcode"].unique()):
    # Creates local dataframe with entries for postcode
    local_df = df[df["Postcode"] == postcode]
    # Summarize postcode, borough and neighbourhood
    new_postcode = ",".join(set(local_df["Postcode"]))
    new_borough = ",".join(set(local_df["Borough"]))
    new_neighbourhood = ",".join(set(local_df["Neighbourhood"]))

    # Use the data generated right above to fill new dataframe new_df
    new_df[index] = new_postcode, new_borough, new_neighbourhood

new_df = new_df.transpose()
new_df.columns = df.columns
print(new_df.shape)

new_df.head(20)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M4V,Central Toronto,"Deer Park,Summerhill West,Rathnelly,South Hill..."
1,M4T,Central Toronto,"Moore Park,Summerhill East"
2,M4S,Central Toronto,Davisville
3,M5N,Central Toronto,Roselawn
4,M5R,Central Toronto,"North Midtown,The Annex,Yorkville"
5,M5P,Central Toronto,"Forest Hill West,Forest Hill North"
6,M4N,Central Toronto,Lawrence Park
7,M4R,Central Toronto,North Toronto West
8,M4P,Central Toronto,Davisville North
9,M4W,Downtown Toronto,Rosedale


# Task nr 4

In [148]:
# Looks up the coordinates of a neighbourhood + city and returns coordinates
def coordinate_search(row):
    address = "{}, Toronto".format(row["Neighbourhood"].split(",")[0])
    location = geocoder.osm(address)
    try:
        latitude, longitude = location.latlng
        return "{},{}".format(latitude, longitude)
    except:
        # print("{} NOT FOUND WITH API".format(address))
        return False

# Creation of Coordinate column
new_df["Coordinates"] = new_df.apply(coordinate_search, axis=1)

# Drop rows that API could not find a latitude and longitude for
new_df = new_df[new_df["Coordinates"] != False]

# Split Coordinate column into latitude and longitude and change type to float
new_df[["Latitude", "Longitude"]] = new_df["Coordinates"].str.split(",", expand= True)
new_df = new_df.drop("Coordinates", axis=1)
new_df[["Latitude", "Longitude"]] = new_df[["Latitude", "Longitude"]].apply(pd.to_numeric, axis=1)
new_df.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4V,Central Toronto,"Deer Park,Summerhill West,Rathnelly,South Hill...",43.688090,-79.394094
1,M4T,Central Toronto,"Moore Park,Summerhill East",43.690388,-79.383297
2,M4S,Central Toronto,Davisville,43.704312,-79.388517
3,M5N,Central Toronto,Roselawn,43.705452,-79.425182
4,M5R,Central Toronto,"North Midtown,The Annex,Yorkville",43.162128,-77.619604
5,M5P,Central Toronto,"Forest Hill West,Forest Hill North",43.693559,-79.413902
6,M4N,Central Toronto,Lawrence Park,43.729199,-79.403253
7,M4R,Central Toronto,North Toronto West,43.653963,-79.387207
8,M4P,Central Toronto,Davisville North,43.704312,-79.388517
9,M4W,Downtown Toronto,Rosedale,43.678356,-79.380746


# Intermediate Folium Analysis

In [149]:
# Toronto coordinates
tor_lat, tor_long = geocoder.osm("Toronto, Canada").latlng
folium_map = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

for label, lat, long in zip(new_df["Neighbourhood"], new_df["Latitude"], new_df["Longitude"]):
    # Folium can't display a map on jupyter notebooks if labels incorporate some characters such as '
    label = label.replace("'" , "")
    
    folium.CircleMarker(
        location=(lat, long),
        radius=10,
        popup=label,
        color="black",
        fill_color = "blue",
        fill = True
    ).add_to(folium_map)

folium_map

# Foursquare API (Search and explore venues)

In [150]:
# The code was removed by Watson Studio for sharing.

In [151]:
# Analysis of etobicoke neighbourhood
etobicoke_map = new_df[new_df["Borough"] == "Etobicoke"].reset_index(drop=True)

## Function is supplied with neighbourhoods and their coordinates, then the Foursquare API is called to find
## and explore venues around those coordinates
def all_neighbourhoods(names, latitudes, longitudes, radius=1000):

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):

        end_point = "explore"
        test_url = 'https://api.foursquare.com/v2/venues/{}'.format(end_point)

        params = dict(
            client_id= Api_id,
            client_secret= Api_secret,
            v=datetime.datetime.now().strftime("%Y%m%d"),
            ll='{}, {}'.format(lat, lng),
            # query='coffee', NOT NECESSARY FOR EXPLORE MODE
            limit=20,
            radius=radius
        )

        result = requests.get(url=test_url, params=params).json()
        # print(json.dumps(result, indent=4))
        filtered_items = result["response"]["groups"][0]["items"]
        venues_list.append(
            [(
            name,
            lat,
            lng,
            entry["venue"]["name"],
            entry["venue"]["categories"][0]["name"],
            entry["venue"]["location"]["distance"],
            entry["venue"]["location"]["lat"],
            entry["venue"]["location"]["lng"]) for entry in filtered_items
            ]
        )
    # Fill and name columns of local dataframe
    nearby_venues = pd.DataFrame([item for venue in venues_list for item in venue])
    nearby_venues.columns = ["Neighbourhood", "Neighbourhood_lat", "Neighbourhood_lng",
                             "Venue_Name",  "Venue_Category",
                             "Venue_Distance", "Venue_Latitude", "Venue_Longitude",
                             ]
    return nearby_venues

# Calls above function and creates dataframe with all etobicoke venues
etobicoke_venues = all_neighbourhoods(etobicoke_map["Neighbourhood"],
                   etobicoke_map["Latitude"],
                   etobicoke_map["Longitude"]
                   )


# Printing of information for each neighbourhood
print(etobicoke_venues.shape)
print("Search revealed {} unique categories".format(len(etobicoke_venues["Venue_Category"].unique())))


(183, 8)
Search revealed 79 unique categories


In [152]:
etobicoke_venues
## >>>>>>>>>>>> Foursquare dataframe finished <<<<<<<<<<<<<<<<

,Neighbourhood,Neighbourhood_lat,Neighbourhood_lng,Venue_Name,Venue_Category,Venue_Distance,Venue_Latitude,Venue_Longitude
0,Westmount,43.693640,-79.521043,Weston Lions Arena,Skating Rink,598,43.698707,-79.518548
1,Westmount,43.693640,-79.521043,UP Express - Weston Station,Train Station,946,43.700256,-79.513660
2,Westmount,43.693640,-79.521043,P&M Restaurant,Diner,847,43.701108,-79.518982
3,Westmount,43.693640,-79.521043,LaRose Bakery,Bakery,805,43.686835,-79.524436
4,Westmount,43.693640,-79.521043,Mayflower Chinese Food,Chinese Restaurant,852,43.692753,-79.531566
5,Westmount,43.693640,-79.521043,Shoppers Drug Mart,Pharmacy,855,43.701264,-79.519705
6,Westmount,43.693640,-79.521043,Pizza Pizza,Pizza Place,824,43.700589,-79.517499
7,Westmount,43.693640,-79.521043,Subway,Sandwich Place,843,43.692927,-79.531471
8,Westmount,43.693640,-79.521043,Popeyes Louisiana Kitchen,Fried Chicken Joint,841,43.701045,-79.518921
9,Westmount,43.693640,-79.521043,Pizza Pizza,Pizza Place,955,43.685256,-79.523582


# Data processing to enable further analysis and machine learning

In [153]:
# Processing
etobicoke_onehot = pd.get_dummies(etobicoke_venues["Venue_Category"], prefix="",prefix_sep="")
etobicoke_onehot["Neighbourhood"] = etobicoke_venues["Neighbourhood"]
sorted_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[sorted_columns]
etobicoke_grouped = etobicoke_onehot.groupby("Neighbourhood").mean().reset_index()
etobicoke_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bakery,Bank,Baseball Field,Beer Store,Bookstore,...,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Train Station,University,Vegetarian / Vegan Restaurant
0,"Alderwood,Long Branch",0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
1,"Humbergate,Albion Gardens,Beaumond Heights,Mou...",0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.058824,0.0,0.00,0.0
2,Islington Avenue,0.00,0.00,0.00,0.00,0.00,0.066667,0.066667,0.066667,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
3,"King's Mill Park,Old Mill South,Kingsway Park ...",0.05,0.00,0.00,0.00,0.05,0.000000,0.000000,0.000000,0.05,...,0.0,0.10,0.05,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
4,"Markland Wood,Bloordale Gardens,Eringate,Old B...",0.00,0.00,0.00,0.00,0.00,0.090909,0.090909,0.000000,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
5,"Mimico South,Humber Bay Shores,New Toronto",0.05,0.00,0.00,0.00,0.05,0.000000,0.000000,0.000000,0.00,...,0.0,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
6,"Montgomery Road,The Kingsway,Old Mill North",0.00,0.00,0.00,0.00,0.05,0.000000,0.000000,0.000000,0.00,...,0.0,0.05,0.00,0.05,0.00,0.05,0.000000,0.0,0.00,0.0
7,Northwest,0.00,0.05,0.05,0.05,0.00,0.000000,0.000000,0.000000,0.00,...,0.0,0.05,0.00,0.05,0.05,0.00,0.000000,0.0,0.05,0.1
8,"Richview Gardens,Kingsview Village,Martin Grov...",0.00,0.00,0.00,0.00,0.00,0.100000,0.000000,0.100000,0.00,...,0.1,0.00,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0
9,"West Deane Park,Martin Grove,Cloverdale,Prince...",0.00,0.00,0.00,0.00,0.10,0.000000,0.000000,0.100000,0.00,...,0.0,0.10,0.00,0.00,0.00,0.00,0.000000,0.0,0.00,0.0


In [154]:
# Desired number of top venues for further analysis
number_top_venues = 10

In [155]:
# Showing information per neighbourhood
# Print popular venues in each neighbourhood by frequency

for neighbourhood in etobicoke_grouped["Neighbourhood"]:
    print(">------", neighbourhood, "<------")
    temp_df = etobicoke_grouped[etobicoke_grouped["Neighbourhood"] == neighbourhood]
    # Transpose and drop the neighbourhood row with iloc
    temp_df = temp_df.transpose().reset_index().iloc[1:]
    temp_df.columns = ["Venue", "Frequency"]
    temp_df.sort_values(by="Frequency", ascending=False, inplace=True)
    print(temp_df.head(number_top_venues))
    print("\n")


>------ Alderwood,Long Branch <------
                Venue Frequency
53        Pizza Place       0.1
23     Discount Store       0.1
20  Convenience Store       0.1
38      Grocery Store       0.1
52           Pharmacy       0.1
61     Sandwich Place      0.05
58                Pub      0.05
63      Shopping Mall      0.05
18        Coffee Shop      0.05
64       Skating Rink      0.05


>------ Humbergate,Albion Gardens,Beaumond Heights,Mount Olive,Thistletown,Jamestown,Silverstone,South Steeles <------
                   Venue  Frequency
18           Coffee Shop   0.294118
40  Gym / Fitness Center  0.0588235
37      Greek Restaurant  0.0588235
28  Fast Food Restaurant  0.0588235
66         Smoothie Shop  0.0588235
17    Chinese Restaurant  0.0588235
16                  Café  0.0588235
34                Garden  0.0588235
68            Steakhouse  0.0588235
14           Bus Station  0.0588235


>------ Islington Avenue <------
            Venue  Frequency
45   Liquor Store  0.0666667


In [156]:
# Create dataframe that summarizes all the above information
# Naming and appening columns
indicators = ["st", "nd", "rd"]
columns = ["Neighbourhood"]

for index in np.arange(number_top_venues):
    try:
        columns.append("{}{} most common venue".format(index+1, indicators[index]))
    except:
        columns.append("{}th most common venue".format(index+1))

# Slice dataframe to process for machine learning
def most_common_categories(row, number_of_venues):
    row_categories = row[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[:number_of_venues]

final_df = pd.DataFrame(columns= columns)
final_df["Neighbourhood"] = etobicoke_grouped["Neighbourhood"]

for ind in np.arange(etobicoke_grouped.shape[0]):
    final_df.iloc[ind, 1:] = most_common_categories(etobicoke_grouped.iloc[ind], number_top_venues)

final_df

,Neighbourhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,"Alderwood,Long Branch",Pizza Place,Convenience Store,Pharmacy,Grocery Store,Discount Store,Pool,Pub,Park,Sandwich Place,Shopping Mall
1,"Humbergate,Albion Gardens,Beaumond Heights,Mou...",Coffee Shop,Gym / Fitness Center,Steakhouse,Garden,Japanese Restaurant,Fast Food Restaurant,Electronics Store,Greek Restaurant,Café,Bus Station
2,Islington Avenue,Golf Course,Pizza Place,Pet Store,Park,Coffee Shop,Café,Sandwich Place,Shopping Mall,Liquor Store,Smoothie Shop
3,"King's Mill Park,Old Mill South,Kingsway Park ...",Italian Restaurant,Sushi Restaurant,American Restaurant,Burrito Place,French Restaurant,Food,Fish & Chips Shop,Latin American Restaurant,Falafel Restaurant,Park
4,"Markland Wood,Bloordale Gardens,Eringate,Old B...",Pizza Place,Park,Flower Shop,Baseball Field,Discount Store,Fast Food Restaurant,Grocery Store,Bank,Golf Course,French Restaurant
5,"Mimico South,Humber Bay Shores,New Toronto",Park,American Restaurant,Liquor Store,Pharmacy,Restaurant,Sandwich Place,Seafood Restaurant,Café,Bus Stop,Pizza Place
6,"Montgomery Road,The Kingsway,Old Mill North",Coffee Shop,Dessert Shop,Italian Restaurant,Pool Hall,Indie Movie Theater,Gastropub,Seafood Restaurant,Liquor Store,Burger Joint,Breakfast Spot
7,Northwest,Vegetarian / Vegan Restaurant,Breakfast Spot,University,Concert Hall,Plaza,Poke Place,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop,Park
8,"Richview Gardens,Kingsview Village,Martin Grov...",Pizza Place,Grocery Store,Pharmacy,Coffee Shop,Bank,Beer Store,Supplement Shop,Sandwich Place,Smoothie Shop,Shopping Mall
9,"West Deane Park,Martin Grove,Cloverdale,Prince...",Electronics Store,Park,Convenience Store,Eastern European Restaurant,Bakery,Pharmacy,Sushi Restaurant,Beer Store,Fast Food Restaurant,Discount Store


# Dataframe creation for clustering

In [157]:
# Run clustering on Neighbourhoods
kclusters = 5

etobicoke_cluster = etobicoke_grouped.drop("Neighbourhood", 1)

# Create cluster instance and append the results to final_df
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_cluster)
final_df.insert(1, "Cluster_Label", kmeans.labels_)

# Merge dataframes to provide an overview of all previous analysis
etobicoke_merged = etobicoke_map
etobicoke_merged = etobicoke_merged.join(final_df.set_index("Neighbourhood"), on="Neighbourhood")
etobicoke_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M9P,Etobicoke,Westmount,43.693640,-79.521043,2,Pizza Place,Café,Train Station,Skating Rink,Discount Store,Middle Eastern Restaurant,Diner,Park,Pharmacy,Chinese Restaurant
1,M9R,Etobicoke,"Richview Gardens,Kingsview Village,Martin Grov...",43.684834,-79.548288,2,Pizza Place,Grocery Store,Pharmacy,Coffee Shop,Bank,Beer Store,Supplement Shop,Sandwich Place,Smoothie Shop,Shopping Mall
2,M9V,Etobicoke,"Humbergate,Albion Gardens,Beaumond Heights,Mou...",43.728530,-79.606240,3,Coffee Shop,Gym / Fitness Center,Steakhouse,Garden,Japanese Restaurant,Fast Food Restaurant,Electronics Store,Greek Restaurant,Café,Bus Station
3,M8V,Etobicoke,"Mimico South,Humber Bay Shores,New Toronto",43.603660,-79.493204,0,Park,American Restaurant,Liquor Store,Pharmacy,Restaurant,Sandwich Place,Seafood Restaurant,Café,Bus Stop,Pizza Place
4,M8Y,Etobicoke,"King's Mill Park,Old Mill South,Kingsway Park ...",43.651090,-79.492029,1,Italian Restaurant,Sushi Restaurant,American Restaurant,Burrito Place,French Restaurant,Food,Fish & Chips Shop,Latin American Restaurant,Falafel Restaurant,Park
5,M9B,Etobicoke,"West Deane Park,Martin Grove,Cloverdale,Prince...",43.663199,-79.568568,4,Electronics Store,Park,Convenience Store,Eastern European Restaurant,Bakery,Pharmacy,Sushi Restaurant,Beer Store,Fast Food Restaurant,Discount Store
6,M9A,Etobicoke,Islington Avenue,43.678207,-79.538269,2,Golf Course,Pizza Place,Pet Store,Park,Coffee Shop,Café,Sandwich Place,Shopping Mall,Liquor Store,Smoothie Shop
7,M9C,Etobicoke,"Markland Wood,Bloordale Gardens,Eringate,Old B...",43.631238,-79.585437,0,Pizza Place,Park,Flower Shop,Baseball Field,Discount Store,Fast Food Restaurant,Grocery Store,Bank,Golf Course,French Restaurant
8,M8X,Etobicoke,"Montgomery Road,The Kingsway,Old Mill North",43.648253,-79.517363,3,Coffee Shop,Dessert Shop,Italian Restaurant,Pool Hall,Indie Movie Theater,Gastropub,Seafood Restaurant,Liquor Store,Burger Joint,Breakfast Spot
9,M8W,Etobicoke,"Alderwood,Long Branch",43.601717,-79.545232,2,Pizza Place,Convenience Store,Pharmacy,Grocery Store,Discount Store,Pool,Pub,Park,Sandwich Place,Shopping Mall


In [158]:
# Visualize neighbourhoods and their clustering values
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Create folium map and add circle markers
toronto_clustermap = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

for lat, lng, neigh, clu_lab in zip(etobicoke_merged["Latitude"], etobicoke_merged["Longitude"], etobicoke_merged["Neighbourhood"], etobicoke_merged["Cluster_Label"]):
    neigh = neigh.replace("'", "")
    folium.CircleMarker(location=(lat, lng),
                        radius = 10,
                        popup = "{}, Cluster {}".format(neigh, clu_lab),
                        color = rainbow[clu_lab],
                        fill = True,
                        fill_color = rainbow[clu_lab],
                        fill_opacity = 0.7
                        ).add_to(toronto_clustermap)
toronto_clustermap

# Dataframes for separate clusters


In [159]:
cluster1 = etobicoke_merged[etobicoke_merged["Cluster_Label"] == 0]
cluster1

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
3,M8V,Etobicoke,"Mimico South,Humber Bay Shores,New Toronto",43.603660,-79.493204,0,Park,American Restaurant,Liquor Store,Pharmacy,Restaurant,Sandwich Place,Seafood Restaurant,Café,Bus Stop,Pizza Place
7,M9C,Etobicoke,"Markland Wood,Bloordale Gardens,Eringate,Old B...",43.631238,-79.585437,0,Pizza Place,Park,Flower Shop,Baseball Field,Discount Store,Fast Food Restaurant,Grocery Store,Bank,Golf Course,French Restaurant


In [160]:
cluster2 = etobicoke_merged[etobicoke_merged["Cluster_Label"] == 1]
cluster2

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
4,M8Y,Etobicoke,"King's Mill Park,Old Mill South,Kingsway Park ...",43.651090,-79.492029,1,Italian Restaurant,Sushi Restaurant,American Restaurant,Burrito Place,French Restaurant,Food,Fish & Chips Shop,Latin American Restaurant,Falafel Restaurant,Park
10,M9W,Etobicoke,Northwest,43.653963,-79.387207,1,Vegetarian / Vegan Restaurant,Breakfast Spot,University,Concert Hall,Plaza,Poke Place,Ramen Restaurant,Japanese Restaurant,Bubble Tea Shop,Park


In [161]:
cluster3 = etobicoke_merged[etobicoke_merged["Cluster_Label"] == 2]
cluster3

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M9P,Etobicoke,Westmount,43.693640,-79.521043,2,Pizza Place,Café,Train Station,Skating Rink,Discount Store,Middle Eastern Restaurant,Diner,Park,Pharmacy,Chinese Restaurant
1,M9R,Etobicoke,"Richview Gardens,Kingsview Village,Martin Grov...",43.684834,-79.548288,2,Pizza Place,Grocery Store,Pharmacy,Coffee Shop,Bank,Beer Store,Supplement Shop,Sandwich Place,Smoothie Shop,Shopping Mall
6,M9A,Etobicoke,Islington Avenue,43.678207,-79.538269,2,Golf Course,Pizza Place,Pet Store,Park,Coffee Shop,Café,Sandwich Place,Shopping Mall,Liquor Store,Smoothie Shop
9,M8W,Etobicoke,"Alderwood,Long Branch",43.601717,-79.545232,2,Pizza Place,Convenience Store,Pharmacy,Grocery Store,Discount Store,Pool,Pub,Park,Sandwich Place,Shopping Mall


In [162]:
cluster4 = etobicoke_merged[etobicoke_merged["Cluster_Label"] == 3]
cluster4

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
2,M9V,Etobicoke,"Humbergate,Albion Gardens,Beaumond Heights,Mou...",43.728530,-79.606240,3,Coffee Shop,Gym / Fitness Center,Steakhouse,Garden,Japanese Restaurant,Fast Food Restaurant,Electronics Store,Greek Restaurant,Café,Bus Station
8,M8X,Etobicoke,"Montgomery Road,The Kingsway,Old Mill North",43.648253,-79.517363,3,Coffee Shop,Dessert Shop,Italian Restaurant,Pool Hall,Indie Movie Theater,Gastropub,Seafood Restaurant,Liquor Store,Burger Joint,Breakfast Spot


In [163]:
cluster5 = etobicoke_merged[etobicoke_merged["Cluster_Label"] == 4]
cluster5

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
5,M9B,Etobicoke,"West Deane Park,Martin Grove,Cloverdale,Prince...",43.663199,-79.568568,4,Electronics Store,Park,Convenience Store,Eastern European Restaurant,Bakery,Pharmacy,Sushi Restaurant,Beer Store,Fast Food Restaurant,Discount Store
